In [1]:
#coding: UTF-8
import sys
sys.path.append("/Users/a_yoshino/python_practice/")
import rakus_ml_training as rmt
import pandas as pd
import numpy as np

load success rakus_ml_training


In [2]:
def deta_clean(x_vec):
    """
    標準化

    :param x_vec:
    :return: 標準化されたデータ

    """
    xmean = x_vec.mean()
    xstd = np.std(x_vec)

    zscore = ((x_vec - xmean)/xstd)

    return zscore

In [5]:
def gradient(vec_x, vec_w, vec_t):
    """
    勾配を計算します

    :param vec_x:
    :param vec_w:
    :return:
    """

    vec_y = np.dot(vec_x, vec_w.T)
    
    vec_dev = vec_y - vec_t
#     print(vec_dev)
    
    vec_dcost = np.dot(vec_dev.T, vec_x)

    return 2 * (vec_dcost / vec_x.shape[1])

In [11]:
def train(vec_x, vec_w, vec_t):
    """
    パラメータを更新します

    :param vec_x:
    :param vec_w:
    :return:
    """
    alpha = 0.00001#学習率

    vec_gradient = gradient(vec_x, vec_w, vec_t) * alpha
    new_vec_w = vec_w - vec_gradient

    return new_vec_w


In [7]:
def cal_error(vec_x, df_target, vec_w):
    """
    targetとの誤差を求める

    :param df_target:
    :param vec_y:
    :return:
    """
    vec_y = np.dot(vec_x, vec_w.T)
    vec_dev = vec_y - df_target

#     error = np.mean(vec_dev**2)/2 #標準誤差
    error = np.sum(vec_dev)**2 / vec_x.shape[0]

    return error

In [8]:
def run(df_train, df_target, count):
    """
    回帰の処理を行います

    :param df_train:
    :param df_target:
    :param count:
    :return:
    """

    vec_w = np.ones([1, df_train.shape[1]])

    for i in range(count):
        # パラメータ更新
        vec_w = train(df_train, vec_w, df_target)
        # 誤差算出
        error = cal_error(df_train, df_target, vec_w)
        print(i,"回目:","{:.4f}".format(error))

#         if np.abs(error) < 2:
#             break

    return vec_w

In [9]:
    boston = rmt.boston.get_train_data()  # 試験用データ
    test = rmt.boston.get_test_data()  # 検証用データ
    
    boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TARGET
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1


In [10]:
    start_calam = 'CRIM'
    end_calam = 'LSTAT'


    # 学習データの準備
    train_origin = np.array(boston.loc[:, start_calam: end_calam])  # feature
#     train_list = deta_clean(train_origin)
    
    #教師データの準備
    target_list = np.array(boston.loc[:, 'TARGET']).reshape([len(boston), 1])  # target
    
    # テストデータの準備
    test_origin = np.array(test.loc[:, start_calam: end_calam])
#     test_list = deta_clean(test_origin)


    # バイアスの設定
    train_list = np.concatenate([np.ones([train_origin.shape[0], 1]), train_origin], axis=1)
    test_list = np.concatenate([np.ones([test_origin.shape[0], 1]), test_origin], axis=1)
    
    # 回帰の実施
    vec_w = run(train_list, target_list, 1000)
    print("vec_w:", vec_w)

    # 試験データの回帰
    nd_result = fit(test_list, vec_w)

    #精度
    df_result = pd.DataFrame(data=nd_result, columns=['TARGET'])
    rmt.boston.confirm(df_result)

0 回目: 10022839669971.5156
1 回目: 318164941995202880.0000
2 回目: 10101216139752186576896.0000
3 回目: 320700620401995197632741376.0000
4 回目: 10181840254902197444519286276096.0000
5 回目: 323260605906733474513224928996622336.0000
6 回目: 10263117191209395950896253576816541827072.0000
7 回目: 325841047708281110747583075096238016113410048.0000
8 回目: 10345042972403634822804608093025702089637826134016.0000
9 回目: 328442088109333170216027182876359405009494937358565376.0000
10 回目: 10427622729977558516149907574553661662371024260566685319168.0000
11 回目: 331063891429618680076723211656636548901944571167617959684210688.0000
12 回目: 10510861684076122179697140132724201014703413982350157518053734088704.0000
13 回目: 333706623409476410535735728825108149176367869902630457589224234438098944.0000
14 回目: 10594765096763079326405307310254746145486729778531230489474000638066569510912.0000
15 回目: 336370451112842926966812185360758799758191026781512637866982243682559132793044992.0000
16 回目: 10679338272108144996548510444207861

/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in double_scalars
  del sys.path[0]
/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in subtract
  if sys.path[0] == '':


 nan
645 回目: nan
646 回目: nan
647 回目: nan
648 回目: nan
649 回目: nan
650 回目: nan
651 回目: nan
652 回目: nan
653 回目: nan
654 回目: nan
655 回目: nan
656 回目: nan
657 回目: nan
658 回目: nan
659 回目: nan
660 回目: nan
661 回目: nan
662 回目: nan
663 回目: nan
664 回目: nan
665 回目: nan
666 回目: nan
667 回目: nan
668 回目: nan
669 回目: nan
670 回目: nan
671 回目: nan
672 回目: nan
673 回目: nan
674 回目: nan
675 回目: nan
676 回目: nan
677 回目: nan
678 回目: nan
679 回目: nan
680 回目: nan
681 回目: nan
682 回目: nan
683 回目: nan
684 回目: nan
685 回目: nan
686 回目: nan
687 回目: nan
688 回目: nan
689 回目: nan
690 回目: nan
691 回目: nan
692 回目: nan
693 回目: nan
694 回目: nan
695 回目: nan
696 回目: nan
697 回目: nan
698 回目: nan
699 回目: nan
700 回目: nan
701 回目: nan
702 回目: nan
703 回目: nan
704 回目: nan
705 回目: nan
706 回目: nan
707 回目: nan
708 回目: nan
709 回目: nan
710 回目: nan
711 回目: nan
712 回目: nan
713 回目: nan
714 回目: nan
715 回目: nan
716 回目: nan
717 回目: nan
718 回目: nan
719 回目: nan
720 回目: nan
721 回目: nan
722 回目: nan
723 回目: nan
724 回目: nan
725 回目: nan
726 回目: nan
727 回目: nan

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').